# Pinecone Tabular

Sample with Pinecone vector database for tabular data search.

In [1]:
# Keys are not typed here :D
from dotenv import load_dotenv

load_dotenv('.env')

True

In [4]:
%%time
%%capture

from datasets import load_dataset

data = load_dataset("ashraq/ott-qa-20k", split="train")


CPU times: total: 281 ms
Wall time: 1.67 s


Let's see some entries.

In [5]:
import pandas as pd

tables = []
for doc in data:
    table = pd.DataFrame(doc['data'], columns=doc['header'])
    tables.append(table)

Initialize Retriever

In [6]:
%%capture

import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the table embedding model from huggingface models hub
retriever = SentenceTransformer("deepset/all-mpnet-base-v2-table", device=device)
retriever

Now let's process the tables

In [8]:
def _preprocess_tables(tables: list):
    processed = []
    # loop through all tables
    for table in tables:
        # convert the table to csv and 
        processed_table = "\n".join([table.to_csv(index=False)])
        # add the processed table to processed list
        processed.append(processed_table)
    return processed


Looking at formatted tables

In [9]:
# format all the dataframes in the tables list
processed_tables = _preprocess_tables(tables)
# display the formatted table
print(str(processed_tables[2]))

Level,Team,League,Manager
AAA,Tidewater Tides,International League,Tom Burgess
AA,Jackson Mets,Texas League,John Antonelli
A,Lynchburg Mets,Carolina League,Jack Aker
A,Wausau Mets,Midwest League,Bill Monbouquette
Rookie,Marion Mets,Appalachian League,Al Jackson



In [10]:
import pinecone
from os import environ

# connect to pinecone environment
pinecone.init(api_key=environ.get('PINECONE_API_KEY'),
              environment=environ.get('PINECONE_API_REGION'))

In [11]:
# Creating the index

# you can choose any name for the index
index_name = "table-qa"

# check if the table-qa index exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(index_name,
                          dimension=768,
                          metric="cosine")

# connect to table-qa index we created
index = pinecone.Index(index_name)


Now we're into embedding and upserting.

In [12]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(processed_tables), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(processed_tables))
    # extract batch
    batch = processed_tables[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch).tolist()
    # create unique IDs ranging from zero to the total number of tables in the dataset
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

100%|██████████| 313/313 [09:33<00:00,  1.83s/it]


{'dimension': 768,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 20000}},
 'total_vector_count': 20000}

In [18]:
query = "Where is Antigua and Barbuda?"
# generate embedding for the query
xq = retriever.encode([query]).tolist()
# query pinecone index to find the table containing answer to the query
result = index.query(xq, top_k=1)
result

{'matches': [{'id': '2960', 'score': 0.554292321, 'values': []}],
 'namespace': ''}

In [19]:
id = int(result["matches"][0]["id"])
tables[id].head()

,Name,Subdivisions,Area ( km² ),Population ( 1 July 2005 est . ),Population density ( per km² ),Capital
0,Antigua and Barbuda,Parishes,440,"85,632",195,St. John 's
1,Antigua and Barbuda,Barbuda,161,"1,370",9.65,Codrington
2,Antigua and Barbuda,Redonda,2,0,0,n/a
3,Barbados,Parishes,431,"284,589",660,Bridgetown
4,Dominica,Parishes,754,"72,660",96.3,Roseau


## Tapas for properly do Q and A

In [30]:
from transformers import pipeline, TapasTokenizer, TapasForQuestionAnswering

model_name = "google/tapas-base-finetuned-wtq"
# load the tokenizer and the model from huggingface model hub
tokenizer = TapasTokenizer.from_pretrained(model_name)
model = TapasForQuestionAnswering.from_pretrained(
    model_name, local_files_only=False)
# load the model and tokenizer into a question-answering pipeline
pipe = pipeline("table-question-answering",  model=model,
                tokenizer=tokenizer, device=0)

In [31]:
pipe(table=tables[id], query=query, device=0)

{'answer': 'Antigua and Barbuda',
 'coordinates': [(0, 0)],
 'cells': ['Antigua and Barbuda'],
 'aggregator': 'NONE'}

In [35]:
def query_pinecone(query):
    # generate embedding for the query
    xq = retriever.encode([query]).tolist()
    # query pinecone index to find the table containing answer to the query
    result = index.query(xq, top_k=1)
    # return the relevant table from the tables list
    return tables[int(result["matches"][0]["id"])]

In [36]:
def get_answer_from_table(table, query):
    # run the table and query through the question-answering pipeline
    answers = pipe(table=table, query=query)
    return answers

In [40]:
query = "where can I find piranhas?"
table = query_pinecone(query)
table

,Species,Common name,Puerto Rico
0,Anolis cuvieri,Puerto Rican Giant Anole,X
1,Anolis roosevelti,Culebra Island Giant Anole,
2,Anolis occultus,Puerto Rican Twig Anole,X
3,Anolis evermanni,Emerald Anole,X
4,Anolis stratulus,Barred Anole,X
5,Anolis gundlachi,Yellow-Chinned Anole,X
6,Anolis cristatellus cristatellus,Puerto Rican Crested Anole,X
7,Anolis cristatellus wileyae,Eastern Puerto Rican Crested Anole,
8,Anolis desechensis,Desecheo Anole,
9,Anolis cooki,Cook 's Anole,X


In [41]:
get_answer_from_table(table, query)


{'answer': 'Anolis cuvieri',
 'coordinates': [(0, 0)],
 'cells': ['Anolis cuvieri'],
 'aggregator': 'NONE'}

In [43]:
query = "which scientist is known for phlogiston?"
table = query_pinecone(query)
table.head()

,Year,British,German
0,2000,"J. C. Smith , National Institute for Medical R...","Thomas Jentsch [ de ] , Center for Molecular N..."
1,1999,"Kay Elizabeth Davies , University of Oxford","Günter Schultz [ de ] , Free University of Berlin"
2,1998,"Ron Laskey , University of Cambridge","Michael Frotscher [ de ] , University of Freib..."
3,1997,"Richard S. J. Frackowiak [ de ] , University C...","Arthur Konnerth , Saarland University"
4,1996,"Alan R. Fersht , University of Cambridge","Walter Neupert [ de ] , Ludwig Maximilian Univ..."


In [44]:
get_answer_from_table(table, query)


{'answer': '', 'coordinates': [], 'cells': [], 'aggregator': 'NONE'}

In [45]:
query = "what is the average carbon emission of power stations in australia, canada and germany?"
table = query_pinecone(query)
table.head()

,CO 2 intensity ( kg/kWh ),Power station,Country
0,1.58,"Hazelwood Power Station , Victoria closed 31 M...",Australia
1,1.56,"Edwardsport IGCC , Edwardsport , Indiana , clo...",United States
2,1.27,"Frimmersdorf power plant , Grevenbroich",Germany
3,1.25,"HR Milner Generating Station , Grande Cache , ...",Canada
4,1.18,"C. TG . Portes Gil , Río Bravo",Mexico


In [46]:
get_answer_from_table(table, query)

{'answer': 'AVERAGE > 1.58, 1.27, 1.25',
 'coordinates': [(0, 0), (2, 0), (3, 0)],
 'cells': ['1.58', '1.27', '1.25'],
 'aggregator': 'AVERAGE'}